In [1]:
import xarray as xr
import numpy as np
import os
import pandas as pd

# Define the paths to the input and output files
input_folder = '/scratch/snx3000/amols/data/camchem_data/'
output_folder = '/scratch/snx3000/amols/data/camchem_data/camchem_hourly/'


In [2]:

def interpolate_ds(target_time,ds1,ds2,ds1_time,ds2_time):

    ds1_time = int(ds1_time)
    ds2_time = int(ds2_time)

    # Calculate the weights based on the inverse distance to the target time
    times = xr.DataArray([ds1_time, ds2_time])
    distances = abs(times - int(target_time))
    weights = 1 / distances
    weights /= weights.sum()  # Normalize weights so they sum to 1

    ds1_weighted = ds1*weights[0]
    ds2_weighted = ds2*weights[1]

    # Concatenate the datasets along the new time dimension
    combined = xr.concat([ds1_weighted, ds2_weighted], dim='time')

    # Use dot product to weight and sum across the time dimension
    interpolated_data = combined.sum(dim='time')
    interpolated_data.coords['time'] = int(target_time)
    
    time_coord = interpolated_data.time
    interpolated_data['NO2'] = interpolated_data['NO2'].expand_dims(time=[time_coord])
    interpolated_data['PS'] = interpolated_data['PS'].expand_dims(time=[time_coord])
    interpolated_data['Q'] = interpolated_data['Q'].expand_dims(time=[time_coord])
    interpolated_data['T'] = interpolated_data['T'].expand_dims(time=[time_coord])

    return interpolated_data

def interpolate_camchem(month, start_day, end_day):
    # Loop through each day and each hour to interpolate the data
    # for i in '01', '02', '03', '04', '05','06','07','08','09', 10, 11, 12:  # Days 12 to 15
    for i in range(start_day, end_day):
        day = i
        print(day)
        # for hour in (['01', '02', '03', '04', '05', '07', '08', '09', 10, 11, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23]):  # Hours 10 to 14
        for hour in (['01', '02', '03', '04', '05', '07', '08', '09', 10, 11, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23]):  # Hours 10 to 14
            if int(hour) < 6:
                ds1_time = '00'
                ds2_time = '06'
            if int(hour) > 6 and int(hour)<12:
                ds1_time = '06'
                ds2_time = 12
            if int(hour) > 12 and int(hour)<18:
                ds1_time = '12'
                ds2_time = 18
            if int(hour) > 18:
                ds1_time = 18
                ds2_time = '00'
                day=str(int(i) + 1).zfill(2)
                                
            ds1 = xr.open_dataset(f"{input_folder}/camchem_20190{month}{day}{ds1_time}_icon.nc")
            ds2 = xr.open_dataset(f"{input_folder}/camchem_20190{month}{day}{ds2_time}_icon.nc")

            interpolated_data = interpolate_ds(hour,ds1,ds2,ds1_time,ds2_time)

            output_file = f"{output_folder}/camchem_20190{month}{day}{hour}_icon.nc"

            # Check if the file already exists
            if os.path.exists(output_file):
                os.remove(output_file)  # Remove the existing file

            interpolated_data.to_netcdf(output_file)
            

In [3]:

# def interpolate_ds(target_time,ds1,ds2,ds1_time,ds2_time):

#     ds1_time = int(ds1_time)

#     # Calculate the weights based on the inverse distance to the target time
#     times = xr.DataArray([ds1_time, ds2_time])
#     distances = abs(times - target_time)
#     weights = 1 / distances
#     weights /= weights.sum()  # Normalize weights so they sum to 1

#     ds1_weighted = ds1*weights[0]
#     ds2_weighted = ds2*weights[1]

#     # Concatenate the datasets along the new time dimension
#     combined = xr.concat([ds1_weighted, ds2_weighted], dim='time')

#     # Use dot product to weight and sum across the time dimension
#     interpolated_data = combined.sum(dim='time')
#     interpolated_data.coords['time'] = target_time
    
#     time_coord = interpolated_data.time
#     interpolated_data['NO2'] = interpolated_data['NO2'].expand_dims(time=[time_coord])
#     interpolated_data['PS'] = interpolated_data['PS'].expand_dims(time=[time_coord])
#     interpolated_data['Q'] = interpolated_data['Q'].expand_dims(time=[time_coord])
#     interpolated_data['T'] = interpolated_data['T'].expand_dims(time=[time_coord])
#     interpolated_data['Z3'] = interpolated_data['Z3'].expand_dims(time=[time_coord])

#     return interpolated_data

# def interpolate_camchem(month):
#     # Loop through each day and each hour to interpolate the data
#     for day in range(12, 25):  # Days 12 to 15
#         for hour in ([10, 11, 13, 14, 15]):  # Hours 10 to 14
#             if hour < 12:
#                 ds1_time = '06'
#                 ds2_time = 12
#             if hour > 12:
#                 ds1_time = 12
#                 ds2_time = 18
            
#             print(ds1_time)

#             ds1 = xr.open_dataset(f"{input_folder}/camchem_20190{month}{day}{ds1_time}_icon.nc")
#             ds2 = xr.open_dataset(f"{input_folder}/camchem_20190{month}{day}{ds2_time}_icon.nc")

#             interpolated_data = interpolate_ds(hour,ds1,ds2,ds1_time,ds2_time)

#             output_file = f"{output_folder}/camchem_20190{month}{day}{hour}_icon.nc"
#             interpolated_data.to_netcdf(output_file)
            

In [4]:
# interpolate_camchem(7, 24, 30)
# interpolate_camchem(2, 14, 25)
interpolate_camchem(7, 19, 31)


19


In [26]:
camchem1 = xr.open_dataset('/scratch/snx3000/amols/data/camchem_data/camchem_2019021206_icon.nc')
camchem2 = xr.open_dataset('/scratch/snx3000/amols/data/camchem_data/camchem_2019021212_icon.nc')
camchem_interp = xr.open_dataset('/scratch/snx3000/amols/data/camchem_data/camchem_hourly/camchem_2019021210_icon.nc')

In [31]:
camchem1['NO2'][0,:,100].values

array([1.57503450e-08, 1.10660787e-08, 1.07156435e-08, 1.13831717e-08,
       1.23704478e-08, 1.19229107e-08, 1.02917070e-08, 9.66450120e-09,
       8.66916849e-09, 7.40942419e-09, 5.66991432e-09, 3.88580457e-09,
       2.31101227e-09, 1.60529978e-09, 1.05888631e-09, 8.35018998e-10,
       7.35348560e-10, 7.40207229e-10, 8.08085321e-10, 5.77836778e-10,
       4.60465777e-10, 3.00467512e-10, 1.96106298e-10, 1.53324575e-10,
       1.54018964e-10, 1.72530074e-10, 1.30229161e-10, 5.10745751e-11,
       1.64890775e-11, 1.82027258e-11, 3.79720422e-11, 4.29041455e-11,
       4.08237541e-11, 3.63197805e-11, 3.26098593e-11, 2.91917775e-11,
       2.47127735e-11, 2.30893308e-11, 2.46850942e-11, 2.81139296e-11,
       3.32694948e-11, 4.00011933e-11, 4.83082220e-11, 5.70358413e-11,
       6.64745203e-11, 7.71872774e-11, 9.23492116e-11, 1.06773979e-10,
       1.18016832e-10, 1.32035868e-10, 1.43499726e-10, 1.54847343e-10,
       1.69688097e-10, 1.96067995e-10, 2.42174725e-10, 3.21553922e-10],
     

In [33]:
camchem2['NO2'][0,:,100].values

array([8.1333995e-10, 1.5020960e-09, 2.3466371e-09, 2.9841551e-09,
       3.3134739e-09, 3.4133785e-09, 3.1369480e-09, 3.0151914e-09,
       2.9912381e-09, 2.7768237e-09, 2.3500020e-09, 1.8619855e-09,
       1.4358476e-09, 1.0034056e-09, 6.5655270e-10, 4.6543000e-10,
       4.8729604e-10, 4.3530543e-10, 3.0732492e-10, 2.2754335e-10,
       1.5089451e-10, 1.0497031e-10, 8.1219573e-11, 6.9341206e-11,
       6.7434225e-11, 8.1884000e-11, 6.8992312e-11, 2.1120796e-11,
       7.7050562e-12, 3.8937009e-12, 3.2097116e-12, 3.7269706e-12,
       4.7824873e-12, 5.9139269e-12, 8.7000103e-12, 1.3251572e-11,
       1.6501083e-11, 2.1253102e-11, 2.6324345e-11, 3.1987340e-11,
       5.2882115e-11, 8.0514616e-11, 8.1173332e-11, 6.6047910e-11,
       5.5971366e-11, 5.1426627e-11, 4.6009262e-11, 4.5295691e-11,
       5.0764542e-11, 5.8655379e-11, 6.8533582e-11, 8.5060542e-11,
       1.2583760e-10, 1.6128771e-10, 2.0157043e-10, 2.7294556e-10],
      dtype=float32)

In [32]:
camchem_interp['NO2'][0,:,100].values

array([5.79234164e-09, 4.69009024e-09, 5.13630590e-09, 5.78382734e-09,
       6.33246522e-09, 6.24988920e-09, 5.52186770e-09, 5.23162802e-09,
       4.88388160e-09, 4.32102384e-09, 3.45663942e-09, 2.53659183e-09,
       1.72756912e-09, 1.20403698e-09, 7.90663905e-10, 5.88626333e-10,
       5.69980211e-10, 5.36939363e-10, 4.74245051e-10, 3.44307823e-10,
       2.54084929e-10, 1.70136044e-10, 1.19515148e-10, 9.73356627e-11,
       9.62958047e-11, 1.12099358e-10, 8.94045948e-11, 3.11053891e-11,
       1.06330633e-11, 8.66337586e-12, 1.47971551e-11, 1.67860289e-11,
       1.67962429e-11, 1.60492114e-11, 1.66699600e-11, 1.85649736e-11,
       1.92383135e-11, 2.18651781e-11, 2.57779284e-11, 3.06962031e-11,
       4.63445752e-11, 6.70101416e-11, 7.02182953e-11, 6.30438872e-11,
       5.94724177e-11, 6.00135104e-11, 6.14559121e-11, 6.57884534e-11,
       7.31819721e-11, 8.31155422e-11, 9.35222964e-11, 1.08322809e-10,
       1.40454435e-10, 1.72881135e-10, 2.15105193e-10, 2.89148344e-10])